In [1]:
import pandas as pd
import numpy as np
from scipy.stats import norm


In [2]:

# Create a DataFrame from your dataset
data = {
    'Cust_ID': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20],
    'Gender': ['M', 'M', 'M', 'M', 'M', 'M', 'F', 'F', 'F', 'F', 'M', 'M', 'M', 'M', 'F', 'F', 'F', 'F', 'F', 'F'],
    'Income': [40000, 50000, 52500, 80000, 100000, 100000, 42000, 40000, 50000, 62000, 80000, 70000, 65000, 140000, 100000, 100000, 120000, 80000, 82000, 150000],
    'Car_Type': ['Family', 'Sports', 'Sports', 'Sports', 'Sports', 'Sports', 'Sports', 'Sports', 'Sports', 'Luxury', 'Family', 'Family', 'Family', 'Luxury', 'Luxury', 'Luxury', 'Luxury', 'Luxury', 'Luxury', 'Luxury'],
    'Shirt_Size': ['Small', 'Medium', 'Medium', 'Large', 'Extra Large', 'Extra Large', 'Small', 'Small', 'Medium', 'Large', 'Large', 'Extra Large', 'Medium', 'Extra Large', 'Small', 'Small', 'Medium', 'Medium', 'Medium', 'Large'],
    'Class': ['C0', 'C0', 'C0', 'C0', 'C0', 'C0', 'C0', 'C0', 'C0', 'C0', 'C1', 'C1', 'C1', 'C1', 'C1', 'C1', 'C1', 'C1', 'C1', 'C1']
}

df = pd.DataFrame(data)



In [3]:
# Calculate prior probabilities for each class
class_counts = df['Class'].value_counts()
class_probs = class_counts / len(df)
class_counts, class_probs 

(C0    10
 C1    10
 Name: Class, dtype: int64,
 C0    0.5
 C1    0.5
 Name: Class, dtype: float64)

In [4]:
def calculate_conditional_prob(attribute, value, class_label):
    subset = df[df['Class'] == class_label]
    mean = np.mean(subset[attribute])
    std_dev = np.std(subset[attribute])
    prob = norm.pdf(value, loc=mean, scale=std_dev)
    return prob


In [5]:
X1 = {
    'Gender': 'M',
    'Income': 130000,
    'Car_Type': 'Luxury',
    'Shirt_Size': 'Medium'
}

X2 = {
    'Gender': 'F',
    'Income': 42000,
    'Car_Type': 'Sports',
    'Shirt_Size': 'Small'
}

In [13]:

pss_X1 = {}
for class_label in class_probs.index:
    ps = class_probs[class_label]
    for attribute, value in X1.items():
        if attribute != 'Income':
            ps *= (len(df[(df['Class'] == class_label) & (df[attribute] == value)]) + 1) / (class_counts[class_label] + len(df[attribute].unique()))
        else:
            ps *= calculate_conditional_prob(attribute, value, class_label)
    pss_X1[class_label] = ps
pss_X1

{'C0': 2.0640109607181044e-09, 'C1': 3.486373392192318e-07}

In [15]:
pss_X2 = {}
for class_label in class_probs.index:
    ps = class_probs[class_label]
    for attribute, value in X2.items():
        print(attribute)
        if attribute != 'Income':
            ps *= (len(df[(df['Class'] == class_label) & (df[attribute] == value)]) + 1) / (class_counts[class_label] + len(df[attribute].unique()))
        else:
            ps *= calculate_conditional_prob(attribute, value, class_label)
    pss_X2[class_label] = ps

pss_X2

Gender
Income
Car_Type
Shirt_Size
Gender
Income
Car_Type
Shirt_Size


{'C0': 5.002011859678653e-07, 'C1': 8.626461344540808e-09}

In [8]:
predicted_class_X1 = max(pss_X1, key=pss_X1.get)
predicted_class_X2 = max(pss_X2, key=pss_X2.get)

print("X1:", predicted_class_X1)
print("X2  (expected C0):", predicted_class_X2)

X1: C1
X2  (expected C0): C1
